In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import yfinance as yf
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import seaborn as sns

import time
from datetime import datetime
import keyboard


class MarkovChain:
    def __init__(self, transition_matrix):
        """
        Inicializa a cadeia de Markov.

        Parameters:
        - transition_matrix: Uma matriz de transição de Markov.
        - initial_state: O estado inicial da cadeia.
        - score: pontuação da cadeia para um conjunto de valores de treino.
        """
        self.transition_matrix = transition_matrix
        self.current_state = np.array(np.zeros(len(transition_matrix)))
        self.score = 0
        self.cost = 0

    def transition(self):
        """
        Realiza uma transição de estado de acordo com a matriz de transição.
        """

        new_state = self.transition_matrix @ self.current_state
        self.current_state = new_state
    
    
    def sim(self, data):
        return self.transition_matrix @ data
    
   
    #=============================================================================================================
    def procriate(self, epsilon=0.1):
        """
        Cria uma nova cadeia de Markov baseada na cadeia existente.
        Uma pequena alteração aleatória é inserida em k elementos da matriz de transição.
        """
        M = self.transition_matrix.copy()

        # Escolhe uma posição aleatória na matriz
        row_idx = np.random.choice(M.shape[0])
        col_idx = np.random.choice(M.shape[1])

        # Gera uma alteração aleatória proporcional à posição escolhida
        # perturbation = np.random.uniform(epsilon*0.1, epsilon)
        perturbation = epsilon
        
        # Seleciona a coluna que será mudada
        col = M[:,col_idx]
        
        # incrementa a perturbação na linha sortiada
        col[row_idx] += perturbation
        
        # Regulariza a coluna para que sua soma dê 1
        col = col/col.sum()
        M[:,col_idx] = col
        
        return MarkovChain(M)
    #=============================================================================================================
    
    def rms(self, data_in, data_out):
        """
        Retorna o desvio quadrado da simulação de uma transição em relação
        aos dados de entrada e saida fornecidos.
        """
        simu = self.transition_matrix @ data_in
        return np.sqrt((data_out-simu)**2).sum()
    
    def reset_cost(self):
        self.cost = 0
    

In [2]:
# Baixa oss dados diários das principais ações 
# Define as principais ações para o modelo

features = 'ABEV3 B3SA3 ITUB4 PETR4 VALE3 CIEL3 JBSS3 LREN3 BBDC4 ELET3'.split(" ")
n = len(features)

dados = pd.DataFrame([])

for feat in features:
    dados[feat] = yf.download(feat+'.SA',
                          start = '2024-01-01',
                          end = '2024-03-28',
                          interval = '1d')['Adj Close'].abs()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
modelo = MarkovChain(np.loadtxt(r'C:\Users\usuario\OneDrive\Área de Trabalho\programas\ajuste-1-02-2024-003.txt', delimiter=','))

In [4]:
dia = str(datetime.now()).split(' ')[0].split('-')[2]
mes = str(datetime.now()).split(' ')[0].split('-')[1]
ano = str(datetime.now()).split(' ')[0].split('-')[0]

In [5]:
print(dia+'/'+mes+'/'+ano)

06/02/2024


In [6]:


colunas = pd.MultiIndex.from_tuples([('Ticker',''),
                                    ('Fechamento Real', str(int(dia)-1).zfill(2)+'-'+mes), 
                                    ('Fechamento Real', dia.zfill(2)+'-'+mes), 
                                    ('Previsão',dia.zfill(2)+'-'+mes ),
                                    ('Previsão',str(int(dia)+1).zfill(2)+'-'+mes )
                                   ])

In [7]:
colunas
header = pd.MultiIndex.from_tuples([('Nome', ''), ('Idade', '(anos)'), ('Cidade', '')])

In [8]:
resultados = []
for i in range(len(features)):
    fecha_ontem = round(dados.iloc[-2,i],2)
    fecha_hoje = round(dados.iloc[-1,i],2)
    prev_hoje = round(modelo.sim(dados.iloc[-2])[i],2)
    prev_amanha = round(modelo.sim(dados.iloc[-1])[i],2)
    resultados.append([features[i],fecha_ontem, fecha_hoje, prev_hoje, prev_amanha])
    
df = pd.DataFrame(resultados, columns=colunas)


In [9]:
df

Ticker Fechamento Real        Previsão       
                   05-02  06-02    06-02  07-02
0  ABEV3           13.09  13.23    13.86  14.14
1  B3SA3           13.07  13.21    13.43  13.65
2  ITUB4           33.36  34.79    32.80  34.09
3  PETR4           41.21  41.59    36.74  37.16
4  VALE3           65.51  66.67    66.36  67.57
5  CIEL3            5.03   5.23     4.81   4.96
6  JBSS3           23.46  23.75    23.67  24.01
7  LREN3           15.02  15.61    17.18  17.84
8  BBDC4           15.63  16.60    16.20  17.05
9  ELET3           40.83  43.13    41.14  43.34